<a href="https://colab.research.google.com/github/AlexeyFroloff/ML1/blob/main/MathHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
#print(data.DESCR)
df = data.frame
df.head()



,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


# **Step 1** #


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop("target", axis = 1)
y = df["target"]

# **Step 2** #

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state = 42)

# **Step 3** #

In [ ]:
from sklearn.linear_model import LinearRegression


model = LinearRegression()
model.fit(Xtrain, ytrain)

LinearRegression()

# **Step 4** #

In [ ]:
print("MSE train:", mean_squared_error(ytrain, model.predict(Xtrain)))
print("MSE test:", mean_squared_error(ytest, model.predict(Xtest)))

print("r2 train:", r2_score(ytrain, model.predict(Xtrain)))
print("r2 test:", r2_score(ytest, model.predict(Xtest)))

MSE train: 2924.0463790726394
MSE test: 2821.7509810013094
r2 train: 0.5244124363545944
r2 test: 0.47728971643226203


# **Step 5** #

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression


from sklearn.model_selection import ShuffleSplit

#cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

metrics = ['neg_mean_absolute_error', 'r2']
model = LinearRegression()

scores = cross_validate(model, X, y, cv=5, scoring=metrics, return_train_score = True)

print(scores["test_r2"].mean())

0.48231643590864215


# **Step 6** #

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
#GridSearchCV

lasso = Lasso()
alpha_s = {"alpha": list(round(0.1*i, 3) for i in range(1, 12))}
model_grid = GridSearchCV(lasso, alpha_s)
model_grid.fit(Xtrain, ytrain)

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0, 1.1]})

# **Step 7** #

In [ ]:
print("best params:", model_grid.best_params_)
print("best estimator:", model_grid.best_estimator_)

best params: {'alpha': 0.1}
best estimator: Lasso(alpha=0.1)


# **Step 8** #

In [ ]:
lasso = Lasso(alpha = 0.1)
lasso.fit(Xtrain, ytrain)
print("r2:", r2_score(ytest, lasso.predict(Xtest)))

r2: 0.4859194402036221


# **Step 9** #

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

model = LinearRegression()
#model = make_pipeline(SplineTransformer(degree = 2), Lasso(alpha = 0.2))
model = Pipeline(steps=[
  #('scaler', StandardScaler()),
  ('preprocessor', PolynomialFeatures(degree=2)),
  ('model', LinearRegression())
])

r2_sc = cross_val_score(model, X, y, cv = 5, scoring = "r2")

print(r2_sc.mean())

0.3915024201503967


# **Step 10** #

In [ ]:
model.fit(Xtrain, ytrain)

print("r2 train:", r2_score(ytrain, model.predict(Xtrain)))
print("r2 test:", r2_score(ytest, model.predict(Xtest)))
print("MSE train:", mean_squared_error(ytrain, model.predict(Xtrain)))
print("MSE test:", mean_squared_error(ytest, model.predict(Xtest)))

r2 train: 0.6081125571650723
r2 test: 0.41297706916214294
MSE train: 2409.434446607772
MSE test: 3168.930443947024




    1. Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

    2. Помогла ли L1-регуляризация улучшить качество модели?

    3. Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?


    1. Качество модели среднее, хорошим его назвать нельзя. Но модель не является переобученной, разве что несильно, так как качество на тесте и на трейне примерно одинаковое.

    2. При добавлении регуляризации модель не стала сильно лучше, качество практически не изменилось.

    3. При добавлении полиномов степени 2 качество модели ухудшилось (судя по тесту). Также, модель сильно переобучилась, подстроилась под трейн - качество на трейне стало лучше, а на трейне все стало совсем печально.


## **Задание 2** ##

In [ ]:
sales = pd.read_csv('https://github.com/Murcha1990/ML_math_2022/raw/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BA%D0%B8/sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [ ]:

index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

# **Mean encodings без регуляризации** #

После проделанной технической работы, мы готовы посчитать счетчики для переменной item_id.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.


Способ 1

In [ ]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])


0.4830386988621699


Способ 2

In [ ]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])


0.4830386988621699


In [ ]:
print(all_data)

          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.022222
141495          0       27               0     0.0         0.056834
144968          0       28               0     0.0         0.141176
142661          0       29               0     0.0         0.037383
138947          0       32               0     6.0         1.319042
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         1.556793
10769024       59    22163              33     0.0         0.581395
10769690       59    22164              33     0.0         1.235589
10771216       59    22166              33     0.0         0.295918
10770511       59    22167              33     0.0         1.081081

[10913850 rows x 5 columns]



# **KFold схема (1.25 балла)** #

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

    1. Разбейте данные на 5 фолдов при помощи sklearn.model_selection.KFold с параметром shuffle=False.
    2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по item_id и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на Способ 1 из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

y = all_data["target"]

kf = KFold(n_splits=5, shuffle=False)
kf.get_n_splits(all_data)


for train_index , val_index in kf.split(all_data):
    train_all = all_data.iloc[train_index]
    val_all = all_data.iloc[val_index]

    all_data["item_target_enc"].iloc[val_index] = val_all["item_id"].map(train_all.groupby('item_id')['target'].mean().to_dict())



all_data['item_target_enc'].fillna(0.3343, inplace=True)



/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
corr = np.corrcoef(all_data['target'].values, all_data["item_target_enc"])[0][1]
print(corr)


0.4164590712798811


Ура, ответ совпадает!





# **Leave-one-out схема (1.25 балла)** #

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.

2. Вычесть таргет конкретного объекта и разделить результат на n_objects - 1.

Заметим, что пункт 1. следует сделать для всех объектов. Также заметим, что пункт 2. может быть реализован без циклов for.

Здесь может оказаться полезной функция .transform из Способа 2 из примера.


In [ ]:
sum_targets = all_data.groupby('item_id')['target'].transform('sum')
n_targets = all_data.groupby('item_id')['target'].transform('count')
all_data['item_target_enc']=(sum_targets-all_data['target'])/ (n_targets-1)

print(np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1])

0.4803848311293002



# **Smoothing (1.25 балла)** #

Необходимо реализовать smoothing с alpha = 100

In [ ]:
globalmean = 0.3343
alpha = 100

sum_targets = all_data.groupby('item_id')['target'].transform('sum')
n_targets = all_data.groupby('item_id')['target'].transform('count')
mean_target = all_data.groupby('item_id')['target'].transform('mean')

all_data['item_target_enc']=(mean_target*n_targets + globalmean*alpha)/ (n_targets+alpha)

In [ ]:
print(np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1])

0.4818198797097264



# **Expanding mean схема (1.25 балла)** #

Необходимо реализовать expanding mean схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки
использовать строки от до . Вам будет необходимо воспользоваться pandas функциями cumsum и cumcount.

In [ ]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

In [ ]:
sum = all_data.groupby('item_id')["target"].cumsum()-all_data["target"]
count = all_data.groupby('item_id')["target"].cumcount()

all_data['item_target_enc'] = sum/count

all_data['item_target_enc'].fillna(0.3343, inplace=True)

In [ ]:
print(np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1])

0.5025245211081697
